In [1]:
using Base.Threads
using BenchmarkTools
using Distributions
using PrettyPrinting
using QuadGK
using Random
using RCall
using Roots
using StatsBase
using StatsFuns
using StatsPlots
default(fmt=:png, size=(400, 250),
    titlefontsize=10, guidefontsize=8, tickfontsize=6)

x ⪅ y = x < y || x ≈ y
x ⪆ y = x > y || x ≈ y
safemul(x, y) = x == 0 ? x : isinf(x) ? typeof(x)(Inf) : x*y
safediv(x, y) = x == 0 ? x : isinf(y) ? zero(y) : x/y

safediv (generic function with 1 method)

In [2]:
"""
    nextcombination!(n, t, c = typeof(t)[min(t-1, i) for i in 1:t])

`[1,2,…,n]` からの重複無しの `t` 個の組み合わせ `c` をすべて生成したい.

`nextcombination!(n, t, c)` は配列で表現された組み合わせ `c` をその次の組み合わせに書き換えて, `c` を返す.

初期条件を `c = typeof(t)[min(t-1, i) for i in 1:t]` にすると, `binomial(n, t)` 回の `nextcombination!(n, t, c)` ですべての組み合わせが生成される.
"""
function nextcombination!(n, t, c = typeof(t)[min(t-1, i) for i in 1:t])
    t == 0 && return c
    @inbounds for i in t:-1:1
        c[i] += 1
        c[i] > (n - (t - i)) && continue
        for j in i+1:t
            c[j] = c[j-1] + 1
        end
        break
    end
    c
end

"""
    mycombinations!(n::Integer, t, c)

事前に割り当てられた組み合わせを格納する配列 `c` を使って, `[1,2,…,n]` からの重複無しの `t` 個の組み合わせのすべてを生成する生成子を返す.
"""
function mycombinations!(n::Integer, t, c)
    for i in 1:t c[i] = min(t - 1, i) end
    (nextcombination!(n, t, c) for _ in 1:binomial(n, t))
end

"""
    mycombinations!(a, t, c)

事前に割り当てられた組み合わせを格納する配列 `c` を使って, 配列 `a` からのインデックスに重複がない `t` 個の組み合わせのすべてを生成する生成子を返す.
"""
function mycombinations!(a, t, c)
    t < 0 && (t = length(a) + 1)
    (view(a, indices) for indices in mycombinations!(length(a), t, c))
end

"""
    mycombinations(x, t)

`x` が整数ならば `[1,2,…,x]` からの, `x` が配列ならば `x` からのインデックスに重複がない `t` 個の組み合わせのすべてを生成する生成子を返す.
"""
mycombinations(x, t) = mycombinations!(x, t, Vector{typeof(t)}(undef, t))

mycombinations

In [3]:
"""
    prob_x_le_y(distx::UnivariateDistribution, disty::UnivariateDistribution;
        a = 0.0)

この函数は, 連続分布 `distx`, `disty` と実数 `a` について, 
`distx` と `disty` に従って生成される乱数をそれぞれ X, Y と書くとき, 
X ≤ Y + a が成立する確率を返す.
"""
function prob_x_le_y(distx::UnivariateDistribution, disty::UnivariateDistribution,
        a = 0.0)
    H(y) = cdf(distx, y) * pdf(disty, y-a)
    quadgk(H, extrema(disty + a)...)[1]
end

"""
    tieshift(distx::UnivariateDistribution, disty::UnivariateDistribution;
        p = 0.5)

この函数は, 連続分布 `distx`, `disty` と実数 `p` について, 
`distx` と `disty` に従って生成される乱数をそれぞれ X, Y と書くとき, 
X ≤ Y + a が成立する確率が `p` に等しくなるような実数 a を返す.
"""
function tieshift(distx::UnivariateDistribution, disty::UnivariateDistribution;
        p=0.5)
    find_zero(a -> prob_x_le_y(distx, disty, a) - p, 0.0)
end

@show tieshift(Normal(0, 1), Normal(2, 2))
@show tieshift(Normal(0, 1), Laplace(2, 2))
@show tieshift(Normal(0, 1), Uniform(0, 1));

tieshift(Normal(0, 1), Normal(2, 2)) = -1.999999999999923
tieshift(Normal(0, 1), Laplace(2, 2)) = -1.99999999999945
tieshift(Normal(0, 1), Uniform(0, 1)) = -0.4999999999999999


In [4]:
"""
    h_brunner_munzel(x, y)

この函数は, x < y のとき 1.0 を, x = y のとき 0.5 を, それら以外のとき 0.0 返す.
"""
h_brunner_munzel(x, y) = (x < y) + (x == y)/2

@doc raw"""
    phat_brunner_munzel(X, Y)

まず以下のようにおく:

```math
\begin{aligned}
&
H(x, y) = \begin{cases} 1 & (x < y) \\ 1/2 & (x = y) \\ 0 & (x > y), \end{cases}
\\ &
m = \mathrm{length}(X), \quad
n = \mathrm{length}(Y), \quad
x_i = X[i], \quad
y_j = Y[j]
\end{aligned}
```

この函数は次の $\hat{p}$ を返す:

```math
\hat{p} = \frac{1}{mn}\sum_{i=1}^m \sum_{j=1}^n H(x_i, y_j).
```
"""
phat_brunner_munzel(X, Y) = mean(h_brunner_munzel(x, y) for x in X, y in Y)

@doc raw"""
    statistics_brunner_munzel(X, Y,
        Hx = similar(X, Float64),
        Hy = similar(Y, Float64);
        p = 1/2
    )

この函数はデータ `X`, `Y` について, Brunner-Munzel検定関係の統計量達を計算する. 詳細は以下の通り.

函数 $H(x, y)$ と $\hat{p}$, $H^x_i$, $H^y_j$, $\bar{H}^x$, $\bar{H}^y$ を次のように定める:

```math
\begin{aligned}
&
m = \mathrm{length}(X), \quad
n = \mathrm{length}(Y), \quad
x_i = X[i], \quad
y_j = Y[j],
\\ &
\hat{p} = \frac{1}{mn}\sum_{i=1}^m \sum_{j=1}^n H(x_i, y_j),
\\ &
H(x, y) = \begin{cases} 1 & (x < y) \\ 1/2 & (x = y) \\ 0 & (x > y), \end{cases}
\\ &
H^x_i = \sum_{j=1}^n H(y_j, x_i), \quad
H^y_j = \sum_{i=1}^m H(x_i, y_j),
\\ &
\bar{H}^x = \frac{1}{m} \sum_{i=1}^m H^x_i = n - n\hat{p},
\\ &
\bar{H}^y = \frac{1}{n} \sum_{j=1}^n H^y_j = m\hat{p}.
\end{aligned}
```

この函数は以下達の named tuple で返す:

```math
\begin{aligned}
&
\mathrm{phat} = 
\hat{p} = \frac{\bar{H}^y - \bar{H}^x + n}{m + n},
\\ &
\mathrm{sx2} =
\hat{\sigma}_x^2 = \frac{1}{n^2}\frac{1}{m-1}\sum_{i=1}^m (H^x_i - \bar{H}^x)^2,
\\ &
\mathrm{sy2} =
\hat{\sigma}_y^2 = \frac{1}{m^2}\frac{1}{n-1}\sum_{j=1}^n (H^y_j - \bar{H}^y)^2,
\\ &
\mathrm{sehat} = 
\widehat{\mathrm{se}} = \sqrt{\frac{\hat{\sigma}_x^2}{m} + \frac{\hat{\sigma}_y^2}{n}}, 
\\ &
\mathrm{tvalue} = t = \frac{\hat{p} - p}{\widehat{\mathrm{se}}},
\\ &
\mathrm{df} =
\nu = 
\frac
{\left(\hat{\sigma}_x^2/m + \hat{\sigma}_y^2/n\right)^2}
{
\dfrac{\left(\hat{\sigma}_x^2/m\right)^2}{m-1} +
\dfrac{\left(\hat{\sigma}_y^2/n\right)^2}{n-1}
},
\\ &
\mathrm{pvalue} =
2\mathrm{ccdf}(\mathrm{TDist}(\nu), |t|).
\end{aligned}
```
"""
function statistics_brunner_munzel(X, Y,
        Hx = similar(X, Float64),
        Hy = similar(Y, Float64);
        p = 1/2
    )
    m, n = length(X), length(Y)
    for (i, x) in pairs(X)
        Hx[i] = sum(h_brunner_munzel(y, x) for y in Y)
    end
    for (j, y) in pairs(Y)
        Hy[j] = sum(h_brunner_munzel(x, y) for x in X)
    end
    phat = (mean(Hy) - mean(Hx) + n)/(m + n)
    sx2, sy2 = var(Hx)/n^2, var(Hy)/m^2
    sehat = √(sx2/m + sy2/n)
    tvalue = (phat - p)/sehat
    df = safediv((sx2/m + sy2/n)^2, (sx2/m)^2/(m-1) + (sy2/n)^2/(n-1))
    pvalue = (df != 0 && isfinite(df)) ? 2ccdf(TDist(df), abs(tvalue)) : zero(df)
    (; phat, sx2, sy2, sehat, tvalue, df, pvalue)
end

@doc raw"""
    pvalue_brunner_munzel(X, Y,
        Hx = similar(X, Float64),
        Hy = similar(Y, Float64);
        p = 1/2
    )

この函数はBrunner-Munzel検定のP値 `pvalue` を返す.
"""
function pvalue_brunner_munzel(X, Y,
        Hx = similar(X, Float64),
        Hy = similar(Y, Float64);
        p = 1/2
    )
    statistics_brunner_munzel(X, Y, Hx, Hy; p).pvalue
end

"""
    tieshift(X::AbstractVector, Y::AbstractVector; p = 1/2)

この函数は `phat_brunner_munzel(X, Y .+ a)` の値が `p` に等しくなる `a` を返す.
"""
function tieshift(X::AbstractVector, Y::AbstractVector; p = 1/2)
    shiftmin = minimum(X) - maximum(Y) - 0.1
    shiftmax = maximum(X) - minimum(Y) + 0.1
    find_zero(a -> phat_brunner_munzel(X, Y .+ a) - p, (shiftmin, shiftmax))
end

@doc raw"""
    brunner_munzel(X, Y,
        Hx = similar(X, Float64),
        Hy = similar(Y, Float64),
        Ytmp = similar(Y, Float64);
        p = 1/2,
        α = 0.05,
        maxsplit = 30
    )

この函数はBrunner-Munzel検定を実行する. 詳細は以下の通り.

この函数は `phat`, `sehat`, `tvalue`, `df`, `p`, `pvalue`, `α` および\
以下達の named tuple を返す.

```math
\begin{aligned}
&
\mathrm{confint\_p} = (\text{$p$ の信頼度 $1-\alpha$ の信頼区間}),
\\ &
\mathrm{confint\_shift} = (\text{2つの集団が互角になるようなシフトの信頼度 $1-\alpha$ の信頼区間}),
\\ &
\mathrm{pvalue\_shift} = ($\mathrm{confint\_shift}$ の計算で使われたP値函数),
\\ &
\mathrm{shifthat} = (\text{2つの集団が互角になるようなシフトの点推定値}).
\end{aligned}
```

さらに, $\mathrm{shiftmin}$, $\mathrm{shiftmax}$ はデータから推定されるシフトの下限と上限.

"""
function brunner_munzel(X, Y,
        Hx = similar(X, Float64),
        Hy = similar(Y, Float64),
        Ytmp = similar(Y, Float64);
        p = 1/2,
        α = 0.05,
        maxsplit = 30
    )
    (; phat, sehat, tvalue, df, pvalue) = statistics_brunner_munzel(X, Y, Hx, Hy; p)
    
    c = df == 0 ? Inf : quantile(TDist(df), 1 - α/2)
    confint_p = [max(0, phat - c*sehat), min(1, phat + c*sehat)]
    
    function pvalue_shift(a)
        @. Ytmp = Y + a
        pvalue_brunner_munzel(X, Ytmp, Hx, Hy; p)
    end
    shiftmin = minimum(X) - maximum(Y) - 0.1
    shiftmax = maximum(X) - minimum(Y) + 0.1
    shifthat = tieshift(X, Y; p)
    confint_shift = [
        find_zero(a -> pvalue_shift(a) - α, (shiftmin, shifthat))
        find_zero(a -> pvalue_shift(a) - α, (shifthat, shiftmax))
    ]
    
    (; phat, sehat, tvalue, df, p, pvalue, α, confint_p,
        confint_shift, pvalue_shift, shifthat, shiftmin, shiftmax)
end

function show_plot_brunner_munzel(X, Y,
        Hx = similar(X, Float64),
        Hy = similar(Y, Float64),
        Ytmp = similar(Y, Float64);
        p = 1/2,
        α = 0.05,
        showXY = false,
        kwargs...
    )
    showXY && (@show X Y)
    (; phat, sehat, tvalue, df, p, pvalue, α, confint_p, 
        confint_shift, pvalue_shift, shifthat, shiftmin, shiftmax) =
        brunner_munzel(X, Y, Hx, Hy, Ytmp; p, α)
    pprint((; phat, sehat, tvalue, df, p, pvalue, α, confint_p,
            confint_shift, shifthat))
    println()
    @show median(X) median(Y)
    plot(pvalue_shift, shiftmin, shiftmax; label="")
    vline!([tieshift(X, Y)]; label="", ls=:dash)    
    title!("P-value function of shift")
    plot!(ytick=0:0.05:1)
    plot!(; kwargs...)
end

show_plot_brunner_munzel (generic function with 4 methods)

In [5]:
"""
    complementcomb!(complcomb::AbstractVector, comb::AbstractVector)

`comb` が {1,2,…,N} から重複無しに m 個を選ぶ組み合わせを表す配列であり, `comb` の中で数は小さな順に並んでいるとし, `complcomb` は長さ N - m の配列であると仮定する.

このとき, この函数は配列 `complcomb` に配列 `comb` の補集合を格納し, `complcomb` を返す.

この函数はメモリ割り当てゼロで実行される.
"""
function complementcomb!(complcomb::AbstractVector, comb::AbstractVector)
    N = length(comb) + length(complcomb)
    k = 0
    a = 0
    @inbounds for b in comb
        for i in a+1:b-1
            k += 1
            complcomb[k] = i
        end
        a = b
    end
    @inbounds for i in a+1:N
        k +=1
        complcomb[k] = i
    end
    complcomb
end

"""
    complementcomb(N, comb::AbstractVector)

`comb` が {1,2,…,N} から重複無しに m 個を選ぶ組み合わせを表す配列であり, `comb` の中で数は小さな順に並んでいると仮定する.

この函数は `comb` の補集合の配列を返す.

この函数は返り値の配列の分だけのメモリ割り当てを行う.
"""
complementcomb(N, comb::AbstractVector) =
    complementcomb!(similar(comb, N - length(comb)), comb)

complementcomb

In [6]:
"""
    permutation_tvalues_brunner_munzel(X, Y,
        XandY = Vector{Float64}(undef, length(X)+length(Y)),
        Tval = Vector{Float64}(undef, binomial(length(X)+length(Y), length(X))),
        Hx = similar(X, Float64),
        Hy = similar(Y, Float64)
    )

Brunner-Munzel検定のt値を `[X; Y]` から\
インデックスの重複無しに `length(X)` 個取る組み合わせと\
その補集合への分割のすべてについて計算して, `Tval` に格納して返す.
"""
function permutation_tvalues_brunner_munzel(X, Y,
        XandY = Vector{Float64}(undef, length(X)+length(Y)),
        Tval = Vector{Float64}(undef, binomial(length(X)+length(Y), length(X))),
        Hx = similar(X, Float64),
        Hy = similar(Y, Float64),
        ccomb = Vector{Int}(undef, length(Y))
    )
    m, n = length(X), length(Y)
    N = m + n
    @views XandY[1:m] .= X
    @views XandY[m+1:N] .= Y
    for (k, comb) in enumerate(mycombinations(1:N, m))
        complementcomb!(ccomb, comb)
        Tval[k] = statistics_brunner_munzel(
            view(XandY, comb), view(XandY, ccomb), Hx, Hy).tvalue
    end
    Tval
end

"""
    pvalue_brunner_munzel_perm(X, Y,
        Tval = permutation_tvalues_brunner_munzel(X, Y),
        tval = statistics_brunner_munzel(X, Y).tvalue;
        le = ⪅
    )

Brunner-Munzel検定のpermutation版のP値を返す.
"""
function pvalue_brunner_munzel_perm(X, Y,
        Tval = permutation_tvalues_brunner_munzel(X, Y),
        tval = statistics_brunner_munzel(X, Y).tvalue;
        le = ⪅
    )
    pvalue_perm = mean(T -> le(abs(tval), abs(T)), Tval)
end

pvalue_brunner_munzel_perm

In [7]:
Random.seed!(4649373)

TaskLocalRNG()

In [8]:
@rlibrary brunnermunzel

In [9]:
distx = Normal(0, 1)
disty = Normal(0, 2)
m = 10
n = 10
L = 30
Xs = [rand(distx, m) for _ in 1:L]
Ys = [rand(disty, n) for _ in 1:L];

In [10]:
pval_R = zeros(0)
for (X, Y) in zip(Xs, Ys)
    o = @time brunnermunzel_permutation_test(X, Y)
    push!(pval_R, rcopy(o)[:p_value])
end
pval_R

  0.277993 seconds (28.99 k allocations: 1.923 MiB, 14.79% compilation time)
  0.233561 seconds (37 allocations: 1008 bytes)
  0.234696 seconds (37 allocations: 1008 bytes)
  0.230429 seconds (37 allocations: 1008 bytes)
  0.240262 seconds (37 allocations: 1008 bytes)
  0.240613 seconds (37 allocations: 1008 bytes)
  0.257392 seconds (37 allocations: 1008 bytes)
  0.239886 seconds (37 allocations: 1008 bytes)
  0.241330 seconds (37 allocations: 1008 bytes)
  0.253003 seconds (37 allocations: 1008 bytes)
  0.240697 seconds (37 allocations: 1008 bytes)
  0.227282 seconds (37 allocations: 1008 bytes)
  0.232373 seconds (37 allocations: 1008 bytes)
  0.234129 seconds (37 allocations: 1008 bytes)
  0.248576 seconds (37 allocations: 1008 bytes)
  0.224460 seconds (37 allocations: 1008 bytes)
  0.228755 seconds (37 allocations: 1008 bytes)
  0.220839 seconds (37 allocations: 1008 bytes)
  0.249647 seconds (37 allocations: 1008 bytes)
  0.212902 seconds (37 allocations: 1008 bytes)
  0.234363 

30-element Vector{Float64}:
 0.6044945766307995
 1.0
 0.7386715451730932
 0.04367923098573254
 0.36544415336985
 0.7943557989997618
 0.2937171187945182
 0.8372989239862305
 0.013195782545627747
 0.7375457359977484
 0.0406157310182078
 0.0615514516443309
 0.4346056420359826
 ⋮
 0.5619086795557384
 0.006884756110762303
 0.4822468553118708
 0.6679079434497391
 0.6774989716166187
 0.7050596462361168
 0.7662646950572647
 0.0693130399012752
 0.7375457359977484
 0.8372989239862305
 0.46709173179761415
 0.09142869514386542

In [11]:
pval_J = zeros(0)
for (X, Y) in zip(Xs, Ys)
    p = @time pvalue_brunner_munzel_perm(X, Y)
    push!(pval_J, p)
end
pval_J

  1.095576 seconds (1.32 M allocations: 112.154 MiB, 1.33% gc time, 86.53% compilation time)
  0.155384 seconds (101.68 k allocations: 30.886 MiB, 10.51% gc time)
  0.160957 seconds (101.68 k allocations: 30.886 MiB, 7.21% gc time)
  0.145725 seconds (101.68 k allocations: 30.886 MiB, 5.06% gc time)
  0.138813 seconds (101.68 k allocations: 30.886 MiB)
  0.144684 seconds (101.68 k allocations: 30.886 MiB, 0.73% gc time)
  0.152213 seconds (101.68 k allocations: 30.887 MiB, 0.90% gc time)
  0.147099 seconds (101.68 k allocations: 30.886 MiB)
  0.140670 seconds (101.68 k allocations: 30.886 MiB, 0.68% gc time)
  0.142780 seconds (101.68 k allocations: 30.886 MiB, 0.59% gc time)
  0.137049 seconds (101.68 k allocations: 30.886 MiB)
  0.142074 seconds (101.68 k allocations: 30.887 MiB, 0.88% gc time)
  0.148235 seconds (101.68 k allocations: 30.886 MiB, 0.62% gc time)
  0.144793 seconds (101.68 k allocations: 30.886 MiB, 0.64% gc time)
  0.140746 seconds (101.68 k allocations: 30.887 MiB)


30-element Vector{Float64}:
 0.6044945766307995
 1.0
 0.7386715451730932
 0.04367923098573254
 0.36544415336985
 0.7943557989997618
 0.2937171187945182
 0.8372989239862305
 0.013195782545627747
 0.7375457359977484
 0.0406157310182078
 0.0615514516443309
 0.4346056420359826
 ⋮
 0.5619086795557384
 0.006884756110762303
 0.4822468553118708
 0.6679079434497391
 0.6774989716166187
 0.7050596462361168
 0.7662646950572647
 0.0693130399012752
 0.7375457359977484
 0.8372989239862305
 0.46709173179761415
 0.09142869514386542

In [12]:
pval_R == pval_J

true

In [13]:
pval_R = zeros(0)
for (X, Y) in zip(Xs[1:10], Ys[1:10])
    o = @btime brunnermunzel_permutation_test($X, $Y)
    push!(pval_R, rcopy(o)[:p_value])
end
pval_R

  235.231 ms (31 allocations: 864 bytes)
  227.450 ms (31 allocations: 864 bytes)
  232.848 ms (31 allocations: 864 bytes)
  225.165 ms (31 allocations: 864 bytes)
  236.751 ms (31 allocations: 864 bytes)
  236.611 ms (31 allocations: 864 bytes)
  242.535 ms (31 allocations: 864 bytes)
  234.456 ms (31 allocations: 864 bytes)
  239.499 ms (31 allocations: 864 bytes)
  244.263 ms (31 allocations: 864 bytes)


10-element Vector{Float64}:
 0.6044945766307995
 1.0
 0.7386715451730932
 0.04367923098573254
 0.36544415336985
 0.7943557989997618
 0.2937171187945182
 0.8372989239862305
 0.013195782545627747
 0.7375457359977484

In [14]:
pval_J = zeros(0)
for (X, Y) in zip(Xs[1:10], Ys[1:10])
    p = @btime pvalue_brunner_munzel_perm($X, $Y)
    push!(pval_J, p)
end
pval_J

  136.410 ms (101677 allocations: 30.89 MiB)
  137.116 ms (101677 allocations: 30.89 MiB)
  136.001 ms (101677 allocations: 30.89 MiB)
  136.880 ms (101677 allocations: 30.89 MiB)
  137.548 ms (101677 allocations: 30.89 MiB)
  136.487 ms (101677 allocations: 30.89 MiB)
  136.935 ms (101679 allocations: 30.89 MiB)
  137.079 ms (101677 allocations: 30.89 MiB)
  135.516 ms (101677 allocations: 30.89 MiB)
  136.510 ms (101677 allocations: 30.89 MiB)


10-element Vector{Float64}:
 0.6044945766307995
 1.0
 0.7386715451730932
 0.04367923098573254
 0.36544415336985
 0.7943557989997618
 0.2937171187945182
 0.8372989239862305
 0.013195782545627747
 0.7375457359977484

In [15]:
pval_R == pval_J

true